In [1]:
import os
from pathlib import Path

print("cwd:", os.getcwd())
print("cwd exists:", Path(os.getcwd()).exists())
print("files in cwd:", list(Path(os.getcwd()).iterdir())[:10])


cwd: c:\Projects\hotel-refund-intelligence\notebooks
cwd exists: True
files in cwd: [WindowsPath('c:/Projects/hotel-refund-intelligence/notebooks/01_ingestion_profiling.ipynb'), WindowsPath('c:/Projects/hotel-refund-intelligence/notebooks/02_cleaning_validation.ipynb'), WindowsPath('c:/Projects/hotel-refund-intelligence/notebooks/03_analysis_storytelling.ipynb')]


In [2]:
from pathlib import Path
import os

# project root = parent of notebooks folder
BASE_DIR = Path.cwd()
if BASE_DIR.name == "notebooks":
    BASE_DIR = BASE_DIR.parent

# If you run notebook from elsewhere, fallback: search upward for README.md
if not (BASE_DIR / "README.md").exists():
    for p in Path.cwd().parents:
        if (p / "README.md").exists():
            BASE_DIR = p
            break

DATA_RAW = BASE_DIR / "data" / "raw"
DATA_INTERIM = BASE_DIR / "data" / "interim"
DATA_PROCESSED = BASE_DIR / "data" / "processed"
FIG_DIR = BASE_DIR / "reports" / "figures"

print("✅ BASE_DIR set to:", BASE_DIR)
print("✅ DATA_RAW:", DATA_RAW)


✅ BASE_DIR set to: c:\Projects\hotel-refund-intelligence
✅ DATA_RAW: c:\Projects\hotel-refund-intelligence\data\raw


In [4]:
import sys
print(sys.executable)


c:\Projects\hotel-refund-intelligence\.venv\Scripts\python.exe


In [7]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

DATA_PATH = PROJECT_ROOT / "data" / "processed" / "refunds_processed.parquet"
df = pd.read_parquet(DATA_PATH)

df.shape, df.head(3)


((63, 8),
        site file_month                   source_file txn_datetime    txn_date  \
 0  Brighton   Dec-2025  Brighton_December_refund.csv   2025-12-26  2025-12-26   
 1  Brighton   Dec-2025  Brighton_December_refund.csv   2025-12-26  2025-12-26   
 2  Brighton   Dec-2025  Brighton_December_refund.csv   2025-12-27  2025-12-27   
 
    refund_amount       BUSINESS_FORMAT_DATE RECEIPT_NO  
 0          150.0  Accomm. Refund/Correction       <NA>  
 1          150.0  Accomm. Refund/Correction       <NA>  
 2         107.99      F&B Refund/Correction       <NA>  )

In [8]:
month_order = ["Nov-2025", "Dec-2025", "Jan-2026"]
df["file_month"] = pd.Categorical(df["file_month"], categories=month_order, ordered=True)


In [9]:
df["refund_amount"] = pd.to_numeric(df["refund_amount"], errors="coerce")
print(df["refund_amount"].dtype)
print("Null rate:", df["refund_amount"].isna().mean())


Float64
Null rate: 0.0


In [10]:
text = df["BUSINESS_FORMAT_DATE"].astype("string").str.lower()

fb_keywords = ["breakfast", "bar", "restaurant", "food", "beverage", "drink", "dinner", "lunch", "meal", "f&b"]
is_fb = text.str.contains("|".join(fb_keywords), na=False)

df["category"] = np.where(is_fb, "F&B", "Accommodation")
df["category"].value_counts()


category
F&B              43
Accommodation    20
Name: count, dtype: int64

In [11]:
kpis = pd.DataFrame([{
    "total_refunds_gbp": df["refund_amount"].sum(skipna=True),
    "refund_count": df["refund_amount"].notna().sum(),
    "avg_refund_gbp": df["refund_amount"].mean(skipna=True)
}])

kpis


,total_refunds_gbp,refund_count,avg_refund_gbp
0,3151.88,63,50.029841


We’ll implement a transparent keyword rule:

If contains food-related keywords → F&B

Else → Accommodation

In [12]:
kpi_by_month = df.groupby("file_month").agg(
    total_refunds_gbp=("refund_amount", "sum"),
    refund_count=("refund_amount", "count"),
    avg_refund_gbp=("refund_amount", "mean")
).reset_index().sort_values("file_month")

kpi_by_month


,file_month,total_refunds_gbp,refund_count,avg_refund_gbp
0,Nov-2025,632.84,9,70.315556
1,Dec-2025,1207.48,19,63.551579
2,Jan-2026,1311.56,35,37.473143


In [13]:
site_month = df.groupby(["site", "file_month"]).agg(
    total_refunds_gbp=("refund_amount", "sum"),
    refund_count=("refund_amount", "count")
).reset_index().sort_values(["file_month","site"])

site_month


,site,file_month,total_refunds_gbp,refund_count
0,Brighton,Nov-2025,231.88,5
3,Newhaven,Nov-2025,400.96,4
1,Brighton,Dec-2025,802.58,7
4,Newhaven,Dec-2025,404.9,12
2,Brighton,Jan-2026,782.94,8
5,Newhaven,Jan-2026,528.62,27


In [14]:
cat_site = df.groupby(["site","category"]).agg(
    count=("refund_amount","count"),
    total_gbp=("refund_amount","sum"),
    avg_gbp=("refund_amount","mean"),
).reset_index()

cat_site


,site,category,count,total_gbp,avg_gbp
0,Brighton,Accommodation,15,1566.54,104.436
1,Brighton,F&B,5,250.86,50.172
2,Newhaven,Accommodation,5,298.99,59.798
3,Newhaven,F&B,38,1035.49,27.249737


In [15]:
df["txn_date"] = pd.to_datetime(df["txn_date"], errors="coerce")

daily = df.groupby(["site", df["txn_date"].dt.date]).agg(
    total_gbp=("refund_amount","sum"),
    count=("refund_amount","count")
).reset_index().rename(columns={"txn_date":"date"})

peak_days = daily.sort_values(["site","total_gbp"], ascending=[True, False]).groupby("site").head(10)
peak_days


,site,date,total_gbp,count
7,Brighton,2025-12-26,300.0,2
10,Brighton,2026-01-10,300.0,3
11,Brighton,2026-01-13,265.97,1
6,Brighton,2025-12-22,137.62,1
1,Brighton,2025-11-23,131.88,3
4,Brighton,2025-12-05,124.99,1
5,Brighton,2025-12-14,120.99,1
8,Brighton,2025-12-27,107.99,1
0,Brighton,2025-11-13,85.0,1
9,Brighton,2026-01-05,63.98,1


In [16]:
HIGH_VALUE_THRESHOLD = 100.0

site_totals = df.groupby("site").agg(
    total_gbp=("refund_amount","sum"),
    total_count=("refund_amount","count"),
).reset_index()

acc = df[df["category"]=="Accommodation"].groupby("site").agg(
    acc_gbp=("refund_amount","sum"),
    acc_avg=("refund_amount","mean"),
).reset_index()

highv = df.assign(is_high=df["refund_amount"]>=HIGH_VALUE_THRESHOLD).groupby("site").agg(
    high_value_share=("is_high","mean")
).reset_index()

sqri = site_totals.merge(acc, on="site", how="left").merge(highv, on="site", how="left")
sqri["acc_share_gbp"] = sqri["acc_gbp"] / sqri["total_gbp"]

sqri


,site,total_gbp,total_count,acc_gbp,acc_avg,high_value_share,acc_share_gbp
0,Brighton,1817.4,20,1566.54,104.436,0.4,0.861968
1,Newhaven,1334.48,43,298.99,59.798,0.023256,0.22405


In [17]:
weights = {
    "acc_share_gbp": 0.45,
    "acc_avg": 0.35,
    "high_value_share": 0.20
}

def minmax(s):
    s = s.astype(float)
    if s.max() == s.min():
        return pd.Series([0.0]*len(s), index=s.index)
    return (s - s.min()) / (s.max() - s.min())

sqri_norm = sqri.copy()
for col in ["acc_share_gbp","acc_avg","high_value_share"]:
    sqri_norm[col+"_norm"] = minmax(sqri_norm[col])

sqri_norm["SQRI"] = (
    weights["acc_share_gbp"]*sqri_norm["acc_share_gbp_norm"] +
    weights["acc_avg"]*sqri_norm["acc_avg_norm"] +
    weights["high_value_share"]*sqri_norm["high_value_share_norm"]
)

sqri_norm[["site","acc_share_gbp","acc_avg","high_value_share","SQRI"]].sort_values("SQRI", ascending=False)


,site,acc_share_gbp,acc_avg,high_value_share,SQRI
0,Brighton,0.861968,104.436,0.4,1.0
1,Newhaven,0.22405,59.798,0.023256,0.0


In [18]:
FIG_DIR = PROJECT_ROOT / "reports" / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR


WindowsPath('c:/Projects/hotel-refund-intelligence/reports/figures')

In [19]:
pivot = site_month.pivot(index="file_month", columns="site", values="total_refunds_gbp").loc[month_order]

ax = pivot.plot(kind="bar")
ax.set_title("Refund Value (£) by Site by Month")
ax.set_xlabel("Month")
ax.set_ylabel("Total Refunds (£)")
plt.tight_layout()

out = FIG_DIR / "refund_value_by_site_by_month.png"
plt.savefig(out, dpi=200)
plt.close()
out


WindowsPath('c:/Projects/hotel-refund-intelligence/reports/figures/refund_value_by_site_by_month.png')

In [20]:
trend = site_month.pivot(index="file_month", columns="site", values="total_refunds_gbp").loc[month_order]

ax = trend.plot(kind="line", marker="o")
ax.set_title("Monthly Refund Trend (£): Brighton vs Newhaven")
ax.set_xlabel("Month")
ax.set_ylabel("Total Refunds (£)")
plt.tight_layout()

out = FIG_DIR / "monthly_refund_trend.png"
plt.savefig(out, dpi=200)
plt.close()
out


WindowsPath('c:/Projects/hotel-refund-intelligence/reports/figures/monthly_refund_trend.png')

In [21]:
cat_value = df.groupby(["site","category"])["refund_amount"].sum().reset_index()
cat_pivot = cat_value.pivot(index="site", columns="category", values="refund_amount")

ax = cat_pivot.plot(kind="bar", stacked=True)
ax.set_title("Refund Value (£) Split: Accommodation vs F&B (by Site)")
ax.set_xlabel("Site")
ax.set_ylabel("Total Refunds (£)")
plt.tight_layout()

out = FIG_DIR / "category_split_by_site_stacked.png"
plt.savefig(out, dpi=200)
plt.close()
out


WindowsPath('c:/Projects/hotel-refund-intelligence/reports/figures/category_split_by_site_stacked.png')

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Ensure numeric
df["refund_amount"] = pd.to_numeric(df["refund_amount"], errors="coerce")
df["txn_date"] = pd.to_datetime(df["txn_date"], errors="coerce")

# Build daily matrix: date x site
d = df.dropna(subset=["txn_date"]).copy()
d["date"] = d["txn_date"].dt.date

dmat = d.groupby(["date", "site"])["refund_amount"].sum(min_count=1).reset_index()
heat = (
    dmat.pivot(index="date", columns="site", values="refund_amount")
        .fillna(0.0)
        .astype(float)
)

# Improve readability: log scale + clip extreme outliers (95th percentile)
vals = heat.to_numpy().ravel()
vals = vals[vals > 0]
vmax = np.quantile(vals, 0.95) if len(vals) else 1.0

data = np.clip(heat.to_numpy(), 0, vmax)

# Plot
plt.figure(figsize=(8, 10))
norm = mcolors.LogNorm(vmin=1, vmax=max(vmax, 2))  # log makes low values visible
im = plt.imshow(data, aspect="auto", norm=norm, cmap="viridis")

# Y ticks: show every Nth date only
dates = [str(x) for x in heat.index]
step = max(1, len(dates)//15)  # ~15 labels
yticks = list(range(0, len(dates), step))
plt.yticks(yticks, [dates[i] for i in yticks])

plt.xticks(range(len(heat.columns)), heat.columns)
plt.title("Daily Refund Heatmap (£) by Site (log scale, clipped)")
plt.xlabel("Site")
plt.ylabel("Date")

# Gridlines
plt.gca().set_xticks(np.arange(-.5, len(heat.columns), 1), minor=True)
plt.gca().set_yticks(np.arange(-.5, len(heat.index), 1), minor=True)
plt.grid(which="minor", linewidth=0.3)
plt.tick_params(which="minor", bottom=False, left=False)

# Colorbar (legend)
cbar = plt.colorbar(im)
cbar.set_label("Daily refund total (£) — log scale (clipped at 95th pct)")

plt.tight_layout()

out = FIG_DIR / "daily_refund_heatmap.png"  # overwrite with improved version
plt.savefig(out, dpi=250)
plt.close()
out



WindowsPath('c:/Projects/hotel-refund-intelligence/reports/figures/daily_refund_heatmap.png')

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

df["refund_amount"] = pd.to_numeric(df["refund_amount"], errors="coerce")
df["txn_date"] = pd.to_datetime(df["txn_date"], errors="coerce")

FIG_DIR = PROJECT_ROOT / "reports" / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

temp = df.dropna(subset=["txn_date"]).copy()
temp["week_start"] = temp["txn_date"] - pd.to_timedelta(temp["txn_date"].dt.weekday, unit="D")  # Monday
temp["dow"] = temp["txn_date"].dt.day_name().str[:3]
dow_order = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
temp["dow"] = pd.Categorical(temp["dow"], categories=dow_order, ordered=True)

daily = temp.groupby(["site","week_start","dow"])["refund_amount"].sum(min_count=1).reset_index()

def plot_weekday_heatmap(site_name: str):
    mat = (
        daily[daily["site"] == site_name]
        .pivot(index="week_start", columns="dow", values="refund_amount")
        .reindex(columns=dow_order)
        .fillna(0.0)
        .astype(float)
    )

    # clip & log for readability
    vals = mat.to_numpy().ravel()
    vals = vals[vals > 0]
    vmax = np.quantile(vals, 0.95) if len(vals) else 1.0
    data = np.clip(mat.to_numpy(), 0, vmax)

    plt.figure(figsize=(11, 4.8))
    norm = mcolors.LogNorm(vmin=1, vmax=max(vmax, 2))
    im = plt.imshow(data, aspect="auto", norm=norm, cmap="viridis")

    plt.xticks(range(len(mat.columns)), mat.columns)
    ylabels = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(mat.index)]
    plt.yticks(range(len(ylabels)), ylabels)

    plt.title(f"Weekly Pattern Heatmap (£) — {site_name} (Week × Day, log scale, clipped)")
    plt.xlabel("Day of Week")
    plt.ylabel("Week starting (Mon)")

    # Gridlines
    plt.gca().set_xticks(np.arange(-.5, len(mat.columns), 1), minor=True)
    plt.gca().set_yticks(np.arange(-.5, len(mat.index), 1), minor=True)
    plt.grid(which="minor", linewidth=0.3)
    plt.tick_params(which="minor", bottom=False, left=False)

    # Colorbar
    cbar = plt.colorbar(im)
    cbar.set_label("Refund total (£) — log scale (clipped at 95th pct)")

    plt.tight_layout()
    out = FIG_DIR / f"heatmap_weekday_{site_name.lower()}.png"
    plt.savefig(out, dpi=250)
    plt.close()
    return out

out_brighton = plot_weekday_heatmap("Brighton")
out_newhaven = plot_weekday_heatmap("Newhaven")

out_brighton, out_newhaven


(WindowsPath('c:/Projects/hotel-refund-intelligence/reports/figures/heatmap_weekday_brighton.png'),
 WindowsPath('c:/Projects/hotel-refund-intelligence/reports/figures/heatmap_weekday_newhaven.png'))

In [24]:
# --- Numbers to paste into markdown deliverables ---
summary = {
    "total_refunds_gbp": float(df["refund_amount"].sum()),
    "refund_count": int(df["refund_amount"].count()),
    "avg_refund_gbp": float(df["refund_amount"].mean()),
}

by_site = (
    df.groupby("site")["refund_amount"]
      .agg(total_refunds_gbp="sum", refund_count="count", avg_refund_gbp="mean")
      .reset_index()
)

by_month = (
    df.groupby("file_month")["refund_amount"]
      .agg(total_refunds_gbp="sum", refund_count="count", avg_refund_gbp="mean")
      .reset_index()
      .sort_values("file_month")
)

sqri_out = sqri_norm[["site","acc_share_gbp","acc_avg","high_value_share","SQRI"]].sort_values("SQRI", ascending=False)

summary, by_site, by_month, sqri_out


({'total_refunds_gbp': 3151.88,
  'refund_count': 63,
  'avg_refund_gbp': 50.02984126984127},
        site  total_refunds_gbp  refund_count  avg_refund_gbp
 0  Brighton             1817.4            20           90.87
 1  Newhaven            1334.48            43       31.034419,
   file_month  total_refunds_gbp  refund_count  avg_refund_gbp
 0   Nov-2025             632.84             9       70.315556
 1   Dec-2025            1207.48            19       63.551579
 2   Jan-2026            1311.56            35       37.473143,
        site  acc_share_gbp  acc_avg  high_value_share  SQRI
 0  Brighton       0.861968  104.436               0.4   1.0
 1  Newhaven        0.22405   59.798          0.023256   0.0)